<a href="https://colab.research.google.com/github/Arif-Kasim1/PIAIC-201/blob/main/201_Project_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -Uq langchain_google_genai langchain-anthropic langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [8]:
import getpass
import os
from google.colab import userdata
from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent


os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
# os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    print(f"Someone Called Me! Multiplying {a} and {b}")
    return a * b
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search, multiply]

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash-exp",
    api_key = userdata.get('GOOGLE_API_KEY')
)
response = llm.invoke([HumanMessage(content="hi!")])
response.content

llm_with_tools = llm.bind_tools(tools)

# response = llm_with_tools.invoke([HumanMessage(content="Hi!")])

# print(f"ContentString: {response.content}")
# print(f"ToolCalls: {response.tool_calls}")

# response = llm_with_tools.invoke([HumanMessage(content="What's the weather in SF? and 2 multiply by 8 is what?")])

# print(f"ContentString: {response.content}")
# print(f"ToolCalls: {response.tool_calls}")



agent_executor = create_react_agent(llm, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

# response = agent_executor.invoke(
#     {"messages": [HumanMessage(content="whats the weather in sf?")]}
# )
# response["messages"]

# response = agent_executor.invoke(
#     {"messages": [HumanMessage(content="what is 2 multiply by 8?")]}
# )
# print(response["messages"])

# # textwrap.fill(response['result'], width=80)
# print(response["messages"])

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")


# for chunk in agent_executor.stream(
#     {"messages": [HumanMessage(content="what 2 * 5 is?")]}
# ):
#     print(chunk)
#     print("----")


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1735330466, 'localtime': '2024-12-27 12:14'}, 'current': {'last_updated_epoch': 1735329600, 'last_updated': '2024-12-27 12:00', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 236, 'wind_dir': 'WSW', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 75, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 11.7, 'windchill_f': 53.1, 'heatindex_c': 12.8, 'heatindex_f': 55.0, 'dewpoint_c': 11.7, 'dewpoint_f': 53.0, 'vis_km': 11.0, 'vis_miles': 6.0, 'uv': 2.0, 'gust_mph': 9.9, 'gust_kph': 15.9}}"}, {'url': 'https://www.msn.com/en-us/weather/

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-290cc3d1-98ec-443a-a1bb-c9196365d123-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': '67332d6a-3e2a-4f1c-98e6-02b05d83fa92', 'type': 'tool_call'}], usage_metadata={'input_tokens': 126, 'output_tokens': 13, 'total_tokens': 139, 'input_token_details': {'cache

{'agent': {'messages': [AIMessage(content='The weather in San Francisco is partly cloudy with a temperature of 15.6°C, which feels like 15.6°C. The wind is from the WSW at 10.4 km/h.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5edc07b6-181f-409a-9453-39282a9e8649-0', usage_metadata={'input_tokens': 954, 'output_tokens': 49, 'total_tokens': 1003, 'input_token_details': {'cache_read': 0}})]}}
----


In [9]:

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="what 2 * 5 is?")]}
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 5.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-190a8f1b-a7e8-4da2-87d0-f8a4b15cc237-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 5.0}, 'id': 'd5fb76cb-d5fa-414c-8f5b-44fc6cbca9e3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 128, 'output_tokens': 3, 'total_tokens': 131, 'input_token_details': {'cache_read': 0}})]}}
----
Someone Called Me! Multiplying 2 and 5
{'tools': {